In [1]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
from pathlib import Path
import pandas as pd

from typing import Optional, Any
import requests
from mlscraper.html import Page
from mlscraper.samples import Sample, TrainingSet
from mlscraper.training import train_scraper
import pickle

In [2]:

from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

In [3]:
es_ache = Elasticsearch("http://elasticsearch-ache:9200")
index = "crawl-cites-small"
# es_index = "crawl-common-cites"


In [4]:
res2 = es_ache.search(
    index=index,
    size=150,
)["hits"]["hits"]

In [12]:
!pip install urllib2

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## MLSCRAPER

In [3]:
def create_scraper(http: str, dict_page: dict) -> Any:
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    resp = requests.get(http, headers= headers)
    assert resp.status_code == 200

    training_set = TrainingSet()
    page = Page(resp.content)

    sample= Sample(page, dict_page)
    training_set.add_sample(sample)
    scraper = train_scraper(training_set)
    return scraper


def create_dict(options: dict) -> dict:
    keys=[]
    for key in options.keys():
        if not options[key]:
            keys.append(key)
    for key in keys:
        options.pop(key)
    return options

def test_page_and_save(http: str, scraper: Any, site: str) -> None:
    resp = requests.get(http, headers=headers)
    assert resp.status_code == 200
    result = scraper.get(Page(resp.content))
    print(result)
    if result:
        with open(site, "wb") as f:
            pickle.dump(scraper, f, protocol=pickle.HIGHEST_PROTOCOL)


## canadianlisted

In [5]:
http = "https://rockyview.canadianlisted.com/pets-and-animals/pied-ball-pythons-available_4173999.html"

In [6]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
resp = requests.get(http, headers= headers)
resp.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<title>Pied ball pythons available for sale in Rocky View, Alberta Classifieds - CanadianListed.com</title>\t\t\r\n<meta content="for sale, Pied ball pythons,Male and Female available and ready to go to any python loving. CanadianListed.com has classifieds in Rocky View, Alberta for pets and animals. Kennel hounds, dogs and all kinds of cats" name="description" />\r\n<meta name="robots" content="index, follow" /><script>(function(){/*\n\n Copyright The Closure Library Authors.\n SPDX-License-Identifier: Apache-2.0\n*/\n\'use strict\';var g=function(a){var b=0;return function(){return b<a.length?{done:!1,value:a[b++]}:{done:!0}}},l=this||self,m=/^[\\w+/_-]+[=]{0,2}$/,p=null,q=function(){},r=function(a){var b=typeof a;if("object"==b)if

In [ ]:
name	description	image	production_data	category	url	title	text	price	currency	seller_name	seller_type	location	product	address_type	addres_location	address_country	seller	ships to

In [7]:
dict_scrap = create_dict({"name": "Pied ball pythons available for sale in Rocky View, Alberta Classifieds - CanadianListed.com",
                         "description": "for sale, Pied ball pythons,Male and Female available and ready to go to any python loving. CanadianListed.com has classifieds in Rocky View, Alberta for pets and animals. Kennel hounds, dogs and all kinds of cats",
                        "location":"Rocky View, Alberta",
                        "ship_to":None,
                        "category": "Pets and Animals",
                        "seller": None,
                        "price": None})
scraper = create_scraper(http, dict_scrap)

In [8]:
test_page_and_save(http="https://rockyview.canadianlisted.com/pets-and-animals/carpet-python_4235249.html", scraper=scraper, site="scraper_canadianlisted")

{'category': 'Pets and Animals', 'name': 'CARPET PYTHON for sale in Rocky View, Alberta Classifieds - CanadianListed.com', 'location': 'Rocky View, Alberta', 'description': 'for sale, Carpet python for sale about 3yrs old. Has never bitten, snake is handled a lot.. CanadianListed.com has classifieds in Rocky View, Alberta for pets and animals. Kennel hounds, dogs and all kinds of cats'}


## PRELOVED

In [9]:
http ="https://www.preloved.co.uk/adverts/show/102966807/lincolnshire-greyhound-trust-greyhounds-needing-loving-homes.html"

In [10]:
http ="https://www.preloved.co.uk/adverts/show/102966807/lincolnshire-greyhound-trust-greyhounds-needing-loving-homes.html"
dict_scrap = create_dict({"name": "Lincolnshire Greyhound Trust greyhounds needing loving homes",
                         "description": "WE CURRENTLY HAVE NO GREYHOUNDS AVAILABLE THAT WE CONSIDER CAT TRAINABLE AND HAVE A CONSIDERABLE WAITING LIST.",
                        "location":"Lincoln, Lincolnshire",
                        "ship_to":None,
                        "seller": "Visit Lincsgreyhounds's profile",
                        "price": "£100"})
scraper = create_scraper(http, dict_scrap)

In [11]:
test_page_and_save(http="https://www.preloved.co.uk/adverts/show/122109996/cat-plush-bag-fluffy-with-purr-collar-and-fluffy-tail.html?link=%2Fclassifieds%2Fclothing-jewellery%2Fbags-purses-wallets", scraper=scraper, site="scraper_preloved")

{'price': '£20', 'location': 'Leeds, West Yorkshire', 'description': 'Beautiful plush black çat zip bag. Fluffy and soft with tail. Length appx 17" Height appx 11" strap adjustable to appx 21"Wear on shoulder or shorten to hand held length. Used once for a couple of hours for fun.', 'seller': "Visit Lizzy or Steve's profile", 'name': 'Cat plush bag fluffy with purr collar and fluffy tail'}


## ukclassifieds

In [12]:
http ="https://www.ukclassifieds.co.uk/for-sale/audio-stereo-hifi/headphones-earbuds-airpods/buy-bulk-kubite-f15-rgb-wired-gaming-headphone-with-microphone-in-uk-i55806"
dict_scrap = create_dict({"name": "Buy Bulk Kubite F15 RGB Wired Gaming Headphone With Microphone in UK",
                         "description": "Kubite F15 RGB Wired Gaming Headphone With Microphone",
                        "location":"Unit 2A Westbury Road, Forest Gate London, E7 8BU UK",
                        "ship_to":None,
                        "seller": "Fonegads UK",
                        "price": "£ 10.00"})
scraper = create_scraper(http, dict_scrap)

In [13]:
test_page_and_save(http="https://www.ukclassifieds.co.uk/pets/puppies-dogs-for-sale/labrador-retriever/labrador-puppies-ready-to-go-i40297",
                   scraper=scraper,
                   site="scraper_ukclassifieds")

{'price': '£ 500.00', 'location': 'A4, London WC2N 5DU, UKAL3 8QEBabertonEdinburghScotland', 'description': "We have 6 adorable Labrador Puppies for sale. We have 4 girls and 2 boys. They have been raised by mum. They have been well socialized with adults, children and other pets. They have been handled plenty so they are really sociable. We are looking for stable, caring forever homes for the pups.\nMum is an amazing member of our family. She's the best.\n\nThey will be vet checked and they had their first vaccinations. We will also have them wormed.....\n\nFor more details contact via whatsapp +447570620489", 'seller': 'Emmanuel Barry', 'name': 'Labrador Puppies Ready To Go'}


## ETSY

In [14]:
http ="https://www.etsy.com/se-en/listing/989551080/scrabble-given-names?click_key=5692a5aa7b738b84a1ea71be07752bc685911b3e%3A989551080&click_sum=96115def&ref=related-6"
dict_scrap = create_dict({"name": "Scrabble given names",
                         "description": None,
                        "location":"Ships from France",
                        "ship_to":None,
                        "seller": "Benjamin Broutin",
                        "price": None})
scraper = create_scraper(http, dict_scrap)

In [15]:
test_page_and_save(http="https://www.etsy.com/se-en/listing/1081235251/super-grandpa-custom-socks-grandpa-socks?external=1&ref=cl_listing_page_recs_listings-2-listing-3&external_collection=NzcyODc5OTZ8NTA5NTcyMzY4&page_type=cl_listing_page&dataset=cs",
                   scraper=scraper,
                   site="scraper_etsy")

{'seller': 'cotébonheur', 'name': 'Super grandpa custom socks, grandpa socks, dad socks, godfather uncle, brother, personalized gift', 'location': 'Ships from France'}


## thetaxidermystore

In [16]:
http= "https://www.thetaxidermystore.com/fawn-pedestal-bust-taxidermy-mount-26358.html"
dict_scrap = create_dict({"description": "Fawn Pedestal Bust Taxidermy Mount For Sale",
                          "name": "Whitetail Deer",
                        "location":None,
                        "ship_to":None,
                        "seller": None,
                        "price": "$1,695.00"})
scraper = create_scraper(http, dict_scrap)

In [18]:
test_page_and_save(http="https://www.thetaxidermystore.com/taxidermy-mounts/alligator-rug-taxidermy-mount-for-sale-27868.html",
                   scraper=scraper,
                   site="scraper_thetaxidermystore")

{'name': 'Alligator', 'description': 'Alligator Rug Taxidermy Mount For Sale', 'price': '$625.00'}


## skullsunlimited

In [28]:
http= "https://www.skullsunlimited.com/collections/all-mammals/products/real-broad-footed-mole-skull-lq-12754"

In [29]:
dict_scrap = create_dict({"name": 'Real Broad-footed Mole Skull',
                         "description": "The broad-footed mole, also known as the California mole, can be found dwelling underground in the moist grasslands and open forests of Northern Mexico, California, Nevada, and Oregon. These mammals are part of the Talpidae family, and they spend most of their lives underground digging for and consuming prey. The California mole is a carnivore, and they eat a variety of invertebrates like earthworms and grubs, and will occasionally consume plant matter to balance their diet.",
                        "location":None,
                        "ship_to":None,
                        "seller": None,
                        "price":None})
scraper = create_scraper(http, dict_scrap)

In [31]:
test_page_and_save(http="https://www.skullsunlimited.com/collections/all-reptiles-amphibians/products/river-cooter-skull",
                   scraper=scraper,
                   site="scraper_skullsunlimited")

{'name': 'Real River Cooter Skull', 'description': 'The river cooter is a species of aquatic turtle native to the southeastern United States. It is a large turtle, typically growing to a shell length of 11-14 inches. River cooters are omnivorous and feed on a variety of aquatic plants, insects, snails, crayfish, and carrion. They are primarily aquatic, but may leave the water to bask in the sun. River cooters are an important species in their ecosystems, providing food for larger predators, such as birds and mammals.'}


## 1stdibs

In [34]:
http = "https://www.1stdibs.com/furniture/tables/side-tables/two-tier-table-drawer-signed-karl-springer/id-f_5091563/"

In [35]:
dict_scrap = create_dict({"name": "Two-Tier Table with Drawer signed Karl Springer",
                         "description": "For Sale on 1stDibs - Two-Tier Table with Drawer signed Karl Springer. Gorgeous, seldom-seen two-tier table in tessellated stone with brass inlays. Karl Springer label in single",
                        "location":"Ft Lauderdale, FL",
                        "ship_to":None,
                        "seller": None,
                        "price":None})
scraper = create_scraper(http, dict_scrap)

In [36]:
test_page_and_save(http="https://www.1stdibs.com/art/photography/black-white-photography/haik-kocharian-light-paris/id-a_16393/",
                   scraper=scraper,
                   site="scraper_1stdibs")

{'name': 'In The Light, Paris', 'location': 'New York, NY', 'description': 'For Sale on 1stDibs - In The Light, Paris, Archival Pigment Print by Haik Kocharian. Offered by Robin Rice Gallery Fine Photography.'}


## trocadero

In [43]:
http ="https://www.trocadero.com/stores/stoneking/items/1474512/Vintage-Chinese-Scholar-Taihu-Lake-Stone"

In [44]:
dict_scrap = create_dict({"name": "A Vintage Chinese Scholar Taihu Lake Stone",
                         "description":"A Vintage Chinese Scholar Taihu Lake Stone for desk appreciation,about 50 years. size:7.87 X 4.33 X 0.99 (height X width X thickness),in good condition",
                          "price":None,
                          "location":"Suzhou",
                        "ship_to":None,
                        "seller": "Stone King"})
scraper = create_scraper(http, dict_scrap)

In [45]:
test_page_and_save(http="https://www.trocadero.com/stores/neart/items/1476438/Aiden-Ripley-etching-Geese-1930-pencil-signed",
                   scraper=scraper,
                   site="scraper_trocadero")

{'seller': 'New England Art Exchange', 'name': 'Aiden Ripley etching, Geese, c. 1930, pencil signed', 'location': 'New England Art Exchange\n\n\nView Seller Profile\r\n    14-20 Depot Street\r\n    Peterborough, NH 03458\n603-784-5311 \n\n\nGuest Book', 'description': 'Aiden Lassell Ripley 1896-1969 American A very fine original etching by American sporting artist Aiden Lassell Ripley, image size 8 3/4 x 13 7/8 inches, c. 1930, pencil signed and titled. Ripley studied at the Boston Museum School under Phillip Hale and Frank Benson, then spent several years traveling throughout Europe. He became quite well known for his landscapes and sporting scenes, and also worked throughout his life in wildlife and endangered species preservation. He was a member of many prestigious artist groups of the 20th century, including the National Academy of Design, the American Watercolor Society, and the Audubon Artists; he served as the president of the Boston Guild of Artists from 1959 to 1969.'}


## glacierwear

In [70]:
http = "https://www.glacierwear.com/black-bear-hide-103309.html"

In [71]:
dict_scrap = create_dict({"name": "Black Bear Hide #103309",
                         "description":"- Origin: Canada", 
                          "price":"649.95",
                          "location":None,
                        "ship_to":None,
                        "seller": None})
scraper = create_scraper(http, dict_scrap)

In [73]:
test_page_and_save(http="https://www.glacierwear.com/black-bear-hide-103309.html",
                   scraper=scraper,
                   site="scraper_glacierwear")

{'name': 'Black Bear Hide #103309', 'description': 'bbr103309\n\n- Origin: Canada\n- Color: Black \n- First Quality Fur: Semi-Heavy, Glossy and Silky, Fall Bear\n- 62 Inches from Tip of Nose to Tip of Tail \n- 73 Inches from Tip of Nose to Rear Paw \n- 30 Inches Across the Belly\n- Complete Life Size - All Paws, Claws and Pads, Ears Turned and Lips Split\n- Should be Suitable for a Life Size Mount by a Competent Taxidermist\n- Good for a Wall Hanger, Crafts, Claws, Garments or Accessories\n- Fresh Commercial Tanned', 'price': '649.95'}


## ebay

In [75]:
http = "https://www.ebay.com/itm/384693477548?hash=item599184a0ac:g:0jsAAOSwTHNiTWNa&amdata=enc%3AAQAIAAAA4GBg9A4c7Z11DQDr0I1DAWh864ozZtXprcOzEjdCkvxi12FY1VCl9UgflOOMVMqWUG0B9InP12Y2H6Ksd4dR4uzPC0C6REnu26XTuPc6hUHzRkQSX0lQwt6E1RQxFiOyOEk94oNDaexr1xgzF1SenL%2FBlgviAW1poF1rXEBsHaoWTvfyYAqZ3OgmNjUEeIKbF7PrnAsySKgkKAMEbdPJ6MYYXxQO7GTJlrmn5OH55%2BLU2nsutqWUPYDBhioMSh3kk%2FM1HJRZ5z5vcMjHOWEwxTfzqO%2B2CL2WixXuJcchmyzg%7Ctkp%3ABk9SR6K694j9YQ"

In [83]:
# dict_scrap = create_dict({"name": "Animal Insect Enamel Pin Lapel Collar Pin Corsage Brooch Women Men Jewelry Gift",
#                          "description":"Enamel", 
#                           "price":"0.99",
#                           "location":"义乌, China",
#                         "ship_to":"Worldwide",
#                         "seller": "yaojie_80"})
# scraper = create_scraper(http, dict_scrap)

SelectorSyntaxError: Malformed attribute selector at position 3
  line 1:
div[data-viewport="{"trackableId":"01GZM9D1F0H20BX31VGJBZW923"}"] span
   ^

In [ ]:
test_page_and_save(http=http,
                   scraper=scraper,
                   site="scraper_ebay")

## usedcalgary

In [56]:
http = "https://www.usedcalgary.com/all/18176415"

In [59]:
# dict_scrap = create_dict({"name": "$25 · ** BBQ Grills **",
#                          "description":"BBQ wire grills : two round 22.5 in. $25. ea. Round with handle 17 in. $20. Rectangle 19in. x 20 in. $10. Small 11 in. x 14 in. $5. Round stand up 13.5in. $20. .** Call only** 403-291-4067.", 
#                           "price":None,
#                           "posted":"September 21, 2012",
#                         "category":"Household",
#                         "seller": None})
# scraper = create_scraper(http, dict_scrap)

In [ ]:
# test_page_and_save(http="http://implora.com/Implora%20Brown%20Shark%20Skin%20Leather%20Hide.html?=st0550",
#                    scraper=scraper,
#                    site="scraper_implora")

In [ ]:

    # seeds.append(f"http://www.antiques-atlas.com/search/?q={pattern}&a_status=&sort=priced&a_antiquesdealers=antique&action=search")
    # seeds.append(f"http://picclick.com/?q={pattern}")
    # seeds.append(f"http://www.boneroom.com/apps/search?q={pattern}")
    # seeds.append(f"http://www.trocadero.com/directory/search.php?keyword={pattern}&archinsearch=yes&click_to_search_now_x=0&click_to_search_now_y=0&category=ALL%20ITEMS&period=No%20Selection&sortfield=calculatedPrice&sortdir=SORT_DESC&start=0&l=60&view=icon")
    # seeds.append(f"http://www.glacierwear.com/catalogsearch/result/index/?ajaxcatalog=true&dir=desc&limit=24&order=price&q={pattern}")
    # seeds.append(f"http://chichesterinc.com/?ss360Query={pattern}")
    # seeds.append(f"https://www.worthpoint.com/inventory/search?max=100&sort=BestMatch&img=true&restrictTo=worldwide&query={pattern}&categories=")
    # seeds.append(f"http://implora.com/searchresult.asp?searchimp={pattern}&submit=Search")
    seeds.append(f"http://www.rubylane.com/ni/search?q={pattern}&sort=pricehigh&style=3&cstyle=1ttp://www.auctionzip.com/cgi-bin/auctionsearch.cgi?txtSearchRadius=0&txtSearchZip=&idxSearchCategory=0&txtSearchKeywords={pattern}&month=&year=&searchPath=head&newsearch=1")
    seeds.append(f"http://www.ioffer.com/search/items/{pattern}?order=price_desc")
    # seeds.append(f"http://www.preloved.co.uk/search?keyword={pattern}&orderBy=priceDesc&location=&lat=&lon=&distance=&sectionId=&minimumPrice=&maximumPrice=&advertType=&advertiserType=&promotionType=")
    seeds.append(f"http://www.usedcalgary.com/classifieds/all?description={pattern}")
    seeds.append(f"https://tradingpost.com.au/search-results/?q={pattern}&filter-location-text=&filter-location-dist=%3E500&cat=&filter-sort-by=Price-Dsc")
    seeds.append(f"http://www.ecrater.com/filter.php?a=1&keywords={pattern}&view=list&sort=price_desc")

In [50]:
# import requests
# from mlscraper.html import Page
# from mlscraper.samples import Sample, TrainingSet
# from mlscraper.training import train_scraper
# import pickle
# # fetch the page to train
ebay = 'https://www.ebay.com/itm/275647232802?epid=27036415710&hash=item402ddafb22:g:rJIAAOSwg8Bj0rtx&amdata=enc%3AAQAIAAAA4JkX1NQwkgZ5cqPVH4AfMcE9FbAOLrErBz%2BGM9OEVDkvzRQ1MjK%2FZQsDMp7%2FyxhKzlqnvhtOVqUsFDDl3ayluXbGUuAwZcZNU%2B1sv7rXMFqoSynCFueDCHlRIPpyATvXbZjsr0OZR8ZZlLiFjBwwZIDjqsdEB66WixJR3AfeJXVjQ4TUSSB7UVeukEjUEAFgoKFK3jXWFqG310raIHM565DyKKfitGhm7oeR%2BgHJGTAyjQnlAWv1%2BLVnnyQx9%2FNtoiBOnCwcSKTQdLVXZaUMwA9TaFv0kavQTLyd9BH2CdtJ%7Ctkp%3ABFBMlsfbmPdh'
resp = requests.get(ebay)
assert resp.status_code == 200

# create a sample for Albert Einstein
training_set = TrainingSet()
page = Page(resp.content)
sample = Sample(page, {'product': "Dyson V11 Animal + Cordless Vacuum | Red | Certified Refurbished", "price": "279.99","location": "Fairfield, Ohio, United States", "ships to": "United States", "seller": "dysonstore"})
training_set.add_sample(sample)

# train the scraper with the created training set
scraper = train_scraper(training_set)


In [51]:
# scrape another page
resp = requests.get('https://www.ebay.com/itm/275727724334?_trkparms=amclksrc%3DITM%26aid%3D1110006%26algo%3DHOMESPLICE.SIM%26ao%3D1%26asc%3D248861%26meid%3D06a47d5529684211831a62bbe6f206f4%26pid%3D101224%26rk%3D2%26rkt%3D5%26sd%3D195718678492%26itm%3D275727724334%26pmt%3D0%26noa%3D1%26pg%3D2047675%26algv%3DDefaultOrganicWebV9BertRefreshRanker%26brand%3DUnbranded&_trksid=p2047675.c101224.m-1')
result = scraper.get(Page(resp.content))
print(result)

{'product': 'Burmese Python Print Genuine Cobra SNAKESKIN SNAKE SKIN Leather Hide Pelt Olive   | eBay', 'seller': 'the-snakeskin-guy', 'ships to': 'Worldwide', 'price': '19.06', 'location': 'CN, China'}


In [28]:
with open("scraper_ebay", "wb") as f:
    pickle.dump(scraper, f, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
with open("scraper", "rb") as f:
    # bytes_io = io.BytesIO(f.read())
    scrap = pickle.load(f)

## Auction zip

In [60]:
ebay = 'https://www.auctionzip.com/cgi-bin/ritem.cgi?itemid=52672&top=1'
resp = requests.get(ebay)
assert resp.status_code == 200

ebay2 = 'https://www.auctionzip.com/auction-lot/5-1862-indian-head-cents-g-vg_99D4C93A87'
resp2 = requests.get(ebay2)
assert resp2.status_code == 200

# create a sample for Albert Einstein
training_set = TrainingSet()
page = Page(resp.content)
page2 = Page(resp2.content)

sample1 = Sample(page, {'name': "1924 CADILLAC SEDANHAS BEEN IN THE FAMILY SINCE NEW","location": "City: CHARLESTOWN", "ships to": "State: RI", "seller": "Bob Ward", "price": "Sold For:"})
# sample2 = Sample(page2, {'product': "(5) 1862 INDIAN HEAD CENTS G/VG","location": "Findlay, OH, US 45840", "ships to": "SHIPPING INFO", "seller": "by Silver City Auctions"})
training_set.add_sample(sample1)
# training_set.add_sample(sample2)
# train the scraper with the created training set
scraper = train_scraper(training_set)

In [61]:

# scrape another page
resp = requests.get("https://www.auctionzip.com/cgi-bin/ritem.cgi?itemid=52750&top=1")
result = scraper.get(Page(resp.content))
print(result)

{'price': 'Sold For: $137,500.00', 'location': 'City: Mt. Vernon', 'ships to': 'State: OH', 'seller': 'Michael Hoffman CAI, Auctioneer', 'name': 'Low mileage 1989 Chevy Blazer'}


In [62]:
with open("scraper_auctionzip", "wb") as f:
    pickle.dump(scraper, f, protocol=pickle.HIGHEST_PROTOCOL)

## https://www.ebid.net/

In [36]:

http="http://www.gumtree.com/p/mens-trainers/nike-air-python-prm-snake-skin/1419067474"
dict_scrap = create_dict({"product": "Nike Air PYTHON PRM Snake Skin",
                         "description": "Nike Air PYTHON PRM Snake Skin",
                        "location":"South East London, London",
                        "ship_to":"South East London, London",
                        "seller": "Don",
                        "price": "£200.00"})

ebay = 'http://www.gumtree.com/p/mens-trainers/nike-air-python-prm-snake-skin/1419067474'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
resp = requests.get(ebay, headers= headers)
assert resp.status_code == 200


# create a sample for Albert Einstein
training_set = TrainingSet()
page = Page(resp.content)


sample = Sample(page, {'name': "Nike Air PYTHON PRM Snake Skin",'description': 'Nike Air PYTHON PRM Snake Skin', "location": "South East London",'ship_to': 'South East London, London', 'seller':'Don', 'price': '£200.00'})
# sample2 = Sample(page2, {'product': "(5) 1862 INDIAN HEAD CENTS G/VG","location": "Findlay, OH, US 45840", "ships to": "SHIPPING INFO", "seller": "by Silver City Auctions"})
training_set.add_sample(sample)
# training_set.add_sample(sample2)
# train the scraper with the created training set
scraper = train_scraper(training_set)

In [60]:
dict_scrap

{'product': 'Nike Air PYTHON PRM Snake Skin',
 'description': 'Nike Air PYTHON PRM Snake Skin',
 'location': 'South East London, London',
 'ship_to': 'South East London, London',
 'seller': 'Don',
 'price': '£200.00'}

In [65]:
# scrape another page
resp = requests.get("https://www.gumtree.com/p/mens-trainers/nike-air-force-1-high-tops/1456283340")
result = scraper.get(Page(resp.content))
print(result)

AssertionError: css rule does not match any node (self.css_rule='.highlights-ritem-info--detail__sale li:nth-child(1)', node=<Page self.soup.name='[document]' classes=None, text=Error Page...>)

In [ ]:
def create_scraper(http: str, dict_page: dict) -> Any:
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    resp = requests.get(http, headers= headers)
    assert resp.status_code == 200

    training_set = TrainingSet()
    page = Page(resp.content)

    sample= Sample(page, dict_page)
    training_set.add_sample(sample)
    scraper = train_scraper(training_set)
    return scraper


In [94]:
resp

<Response [503]>

In [ ]:
### A

In [ ]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

In [ ]:
def create_dict(options: dict) -> dict:
    keys=[]
    for key in options.keys():
        if not options[key]:
            keys.append(key)
    for key in keys:
        options.pop(key)
    return options


In [ ]:
def test_page_and_save(http: str, scraper: Any, site: str) -> None:
    resp = requests.get(http, headers=headers)
    assert resp.status_code == 200
    result = scraper.get(Page(resp.content))
    print(result)
    if result:
        with open(site, "wb") as f:
            pickle.dump(scraper, f, protocol=pickle.HIGHEST_PROTOCOL)
        

In [67]:
http="https://www.gumtree.com/p/mens-trainers/new-nike-tempo-next-flyease-size-uk-10/1455633446"
dict_scrap = create_dict({"name": "New Nike Tempo next% Flyease size UK 10",
                         "description": "Brand new no box Nike Air Tempo Next% Flyease size UK10 trainers. Never been worn. Bought from Sports Direct for £145. Absolutely nothing wrong with them, Trainers are as pictured.",
                        "location":"Lambeth, London",
                        "ship_to":None,
                        "seller": "fixnaflexa",
                        "price": "£65.00"})
scraper = create_scraper(http, dict_scrap)


In [68]:
test_page_and_save(http="https://www.gumtree.com/p/mens-trainers/adidas-adizero-speed-professional-race-singlet-2020/1409382639", scraper=scraper, site="scraper_gumtree")

{'price': '£80.00', 'location': 'Loughborough, Leicestershire', 'description': 'Brand new with tags adidas Adizero race running singlet. Provided for contracted adidas athletes. Spare one here. Size XL. \nHappy to post for FREE.', 'seller': 'Contact Harry', 'name': 'Adidas Adizero Speed Professional Race Singlet 2020'}
